# CatBoost baseline (без pandas) — быстрый и экономный по памяти

Этот ноутбук делает сабмит для multi-label задачи (41 таргет) из `main_features`.

Есть **2 режима** подготовки данных:

1) **MEMORY_SAVER = True (по умолчанию)**  
   - категориальные `cat_feature_*` остаются `int32`, **без FeaturesData**  
   - CatBoost видит их как **числовые** (т.е. без спец-обработки категориальных)  
   - **самый экономный по памяти и самый надёжный** (нет `object`, нет строгих проверок FeaturesData)

2) **MEMORY_SAVER = False**  
   - используем `FeaturesData`, чтобы CatBoost обрабатывал категориальные как категориальные  
   - для этого `cat_feature_*` приводим к строкам (`Utf8`)  
   - может потреблять больше памяти

Файлы должны лежать в текущей директории:
- `train_main_features.parquet`
- `test_main_features.parquet`
- `train_target.parquet`


In [1]:
import gc
import numpy as np
import polars as pl

from catboost import Pool, CatBoostClassifier


## Настройки

In [ ]:
TRAIN_MAIN_PATH = "train_main_features.parquet"
TEST_MAIN_PATH  = "test_main_features.parquet"
TRAIN_TGT_PATH  = "train_target.parquet"

MODEL_PATH = "catboost_multilabel.cbm"
SUBMIT_PATH = "submission.parquet"



# True = максимум экономии памяти (cat как num, без FeaturesData)
# False = корректная обработка cat через FeaturesData (см. ниже отдельную ячейку)
MEMORY_SAVER = True


## Загрузка данных

In [3]:
print("Читаем parquet...")
train = pl.read_parquet(TRAIN_MAIN_PATH)
test  = pl.read_parquet(TEST_MAIN_PATH)
target = pl.read_parquet(TRAIN_TGT_PATH)

print("Train:", train.shape)
print("Test :", test.shape)
print("Target:", target.shape)

cat_features = [c for c in train.columns if c.startswith("cat_feature")]
num_features = [c for c in train.columns if c.startswith("num_feature")]
target_cols  = [c for c in target.columns if c.startswith("target")]

print("cat_features:", len(cat_features))
print("num_features:", len(num_features))
print("targets:", len(target_cols))


Читаем parquet...
Train: (750000, 883)
Test : (250000, 883)
Target: (750000, 42)
cat_features: 67
num_features: 815
targets: 41


## Приведение типов (для памяти/скорости)
- `num_feature_*` -> `Float32`
- `cat_feature_*` -> `Int32` (в MEMORY_SAVER режиме остаются числами)
- таргеты -> `Int8`


In [4]:
train = train.with_columns([
    pl.col(num_features).cast(pl.Float32),
    pl.col(cat_features).cast(pl.Int32),
])
test = test.with_columns([
    pl.col(num_features).cast(pl.Float32),
    pl.col(cat_features).cast(pl.Int32),
])
target = target.with_columns([pl.col(target_cols).cast(pl.Int8)])


## Подготовка Pool (MEMORY_SAVER режим: без FeaturesData)

В этом режиме мы делаем один `float32` numpy-массив **по всем признакам**:
- числовые уже `float32`
- категориальные `int32` будут автоматически приведены в `float32` при сборке общей матрицы
- `cat_features` в Pool не задаём

Плюсы: минимальная память, простота, не ломается на строгих проверках.  
Минусы: CatBoost не использует спец-обработку категориальных (может снизить качество).


In [5]:
if MEMORY_SAVER:
    print("MEMORY_SAVER=True: готовим общий float32 массив (cat как num)...")

    feature_cols = [c for c in train.columns if c != "customer_id"]
    X_train = train.select(feature_cols).to_numpy().astype(np.float32, copy=False)
    y_train = target.select(target_cols).to_numpy().astype(np.int8, copy=False)

    X_test = test.select(feature_cols).to_numpy().astype(np.float32, copy=False)
    test_customer_id = test.select("customer_id")

    del train, test, target
    gc.collect()

    train_pool = Pool(X_train, label=y_train)
    test_pool  = Pool(X_test)

    del X_train, y_train, X_test
    gc.collect()

    print("Pools готовы.")
else:
    print("MEMORY_SAVER=False: см. следующую ячейку (FeaturesData).")


MEMORY_SAVER=True: готовим общий float32 массив (cat как num)...
Pools готовы.


## (Опционально) Подготовка Pool через FeaturesData (CatBoost видит категориальные)

Если `MEMORY_SAVER = False`, выполните эту ячейку **вместо** предыдущей.

Важно:
- `num_feature_data` должен быть строго `np.float32`
- `cat_feature_data` должен быть строковым (`Utf8` / python `str`), иначе появляются ошибки вида:
  - `cat_feature_data element type must be object`
  - `object of type 'int' has no len()`

Этот режим может потреблять больше памяти, но качество обычно лучше.


In [ ]:
if not MEMORY_SAVER:
    from catboost import FeaturesData

    print("MEMORY_SAVER=False: готовим num(float32) + cat(str) через FeaturesData...")

    X_num_train = train.select(num_features).to_numpy().astype(np.float32, copy=False)
    X_cat_train = train.select(cat_features).with_columns(pl.all().cast(pl.Utf8)).to_numpy()
    y_train = target.select(target_cols).to_numpy().astype(np.int8, copy=False)

    X_num_test = test.select(num_features).to_numpy().astype(np.float32, copy=False)
    X_cat_test = test.select(cat_features).with_columns(pl.all().cast(pl.Utf8)).to_numpy()

    test_customer_id = test.select("customer_id")

    del train, test, target
    gc.collect()

    train_fd = FeaturesData(
        num_feature_data=X_num_train,
        cat_feature_data=X_cat_train,
        num_feature_names=num_features,
        cat_feature_names=cat_features,
    )
    test_fd = FeaturesData(
        num_feature_data=X_num_test,
        cat_feature_data=X_cat_test,
        num_feature_names=num_features,
        cat_feature_names=cat_features,
    )

    train_pool = Pool(data=train_fd, label=y_train)
    test_pool  = Pool(data=test_fd)

    del X_num_train, X_cat_train, y_train, X_num_test, X_cat_test
    gc.collect()

    print("Pools готовы.")


## Обучение CatBoost (MultiLogloss)

In [ ]:
model = CatBoostClassifier(
    iterations=2000,
    depth=6,
    learning_rate=0.05,
    loss_function="MultiLogloss",
    eval_metric="MultiLogloss",
    random_seed=42,
    nan_mode="Min",
    thread_count=-1,
    bootstrap_type="Bernoulli",
    subsample=0.8,
    l2_leaf_reg=3.0,
    early_stopping_rounds=100,
    verbose=1
)

model.fit(train_pool)
model.save_model(MODEL_PATH)
print("Model saved:", MODEL_PATH)


0:	learn: 0.5864431	total: 14.3s	remaining: 3h 11m 5s
1:	learn: 0.4994583	total: 28.7s	remaining: 3h 11m 1s
2:	learn: 0.4290234	total: 42.9s	remaining: 3h 10m 4s
3:	learn: 0.3721578	total: 56.9s	remaining: 3h 8m 51s
4:	learn: 0.3262626	total: 1m 10s	remaining: 3h 7m 36s
5:	learn: 0.2891396	total: 1m 24s	remaining: 3h 6m 41s
6:	learn: 0.2590213	total: 1m 38s	remaining: 3h 5m 55s
7:	learn: 0.2344681	total: 1m 52s	remaining: 3h 5m 2s
8:	learn: 0.2143548	total: 2m 5s	remaining: 3h 4m 14s
9:	learn: 0.1977897	total: 2m 19s	remaining: 3h 3m 40s
10:	learn: 0.1840761	total: 2m 33s	remaining: 3h 3m 18s
11:	learn: 0.1726636	total: 2m 46s	remaining: 3h 2m 45s
12:	learn: 0.1631202	total: 3m 1s	remaining: 3h 2m 38s
13:	learn: 0.1550986	total: 3m 15s	remaining: 3h 2m 33s
14:	learn: 0.1483293	total: 3m 29s	remaining: 3h 2m 19s
15:	learn: 0.1425904	total: 3m 42s	remaining: 3h 1m 53s
16:	learn: 0.1377022	total: 3m 56s	remaining: 3h 1m 44s
17:	learn: 0.1335294	total: 4m 12s	remaining: 3h 3m 1s
18:	learn:

KeyboardInterrupt: 

## Предсказания и сохранение submission.parquet

In [ ]:
print("Predicting...")
test_pred = model.predict(test_pool, prediction_type="RawFormulaVal")
print("test_pred shape:", test_pred.shape)

predict_cols = [c.replace("target_", "predict_") for c in target_cols]
pred_df = pl.DataFrame(test_pred, schema=predict_cols)

submit = test_customer_id.hstack(pred_df)
submit.write_parquet(SUBMIT_PATH)

print("Submit saved:", SUBMIT_PATH)
